In [1]:
import jax
import jax.numpy as jnp
import pickle
import sys
sys.path.append('..')
sys.path.append('../analyses')

import analysis
import datatypes
import input_pipeline_tf

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
version = 3
interactions = 4
l = 5

config, best_state_train, best_state_eval, metrics_for_best_state = analysis.load_from_workdir(f'../workdirs/mace/v{version}/interactions={interactions}/l={l}/channels=32')
key = jax.random.PRNGKey(config.rng_seed)

qm9_datasets = input_pipeline_tf.get_datasets(key, config)
# example_graph = next(qm9_datasets["test"].as_numpy_iterator())
# frag = datatypes.Fragments.from_graphstuple(example_graph)
# frag = jax.tree_map(jnp.asarray, frag)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/home/songk/atomicarchitects/spherical_harmonic_net/notebooks/../input_pipeline_tf.py:94: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  np.shape(jax.tree_leaves(graph.nodes)[0])[0],


In [20]:
generated_dict = {}
qm9_iterator = qm9_datasets["test"].as_numpy_iterator()
for _ in range(10):
    example_graph = next(qm9_iterator)
    frag = datatypes.Fragments.from_graphstuple(example_graph)
    frag = jax.tree_map(jnp.asarray, frag)
    analysis.update_dict(generated_dict, analysis.to_mol_dict(frag, False))

In [21]:
datasets = input_pipeline_tf.get_raw_qm9_datasets(
        config.root_dir,
        config.train_molecules,
        config.val_molecules,
        config.test_molecules,
    )

In [23]:
train_iter = datasets['train'].as_numpy_iterator()
a = next(train_iter)

In [26]:
from ase.atoms import Atoms
atoms_a = Atoms(positions=a['positions'], numbers=models.get_atomic_numbers(a['species']))

In [27]:
from ase.db import connect
db = connect('asdf.db')
db.write(atoms_a)

1

In [4]:
with open('../workdirs/generated/generated_molecules.mol_dict', 'wb') as f:
    pickle.dump(generated_dict, f)

In [10]:
mol_dict = pickle.load(open('../workdirs/generated/generated_molecules.mol_dict', 'rb'))